In [15]:
%matplotlib notebook
# Dependencies
from census import Census
from config import (census_key, gkey)
import gmaps
import numpy as np
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt
import json
from pprint import pprint
import os
import random
from itertools import accumulate


# Census API Key
c = Census(census_key, year=2017)

In [16]:
State_FIPS_pd = pd.read_csv("State_FIPS_Codes.csv")
State_FIPS_pd

,State FIPS,State Name
0,1,Alabama
1,2,Alaska
2,4,Arizona
3,5,Arkansas
4,6,California
5,8,Colorado
6,9,Connecticut
7,10,Delaware
8,12,Florida
9,13,Georgia


In [17]:
education_percentages = pd.read_csv("education_percentages.csv")
len(education_percentages)

50

In [18]:
education_merge = pd.merge(State_FIPS_pd, education_percentages, how="right", left_on="State Name", right_on="State Name")
education_merge.head()

,State FIPS,State Name,No College Degree Percentage,Associates or Bachelors Degree Percentage,Post Graduate Degree Percentage
0,1,Alabama,0.647095,0.318372,0.034533
1,2,Alaska,0.617160,0.345856,0.036984
2,4,Arizona,0.596793,0.362131,0.041076
3,5,Arkansas,0.689379,0.279693,0.030928
4,6,California,0.548433,0.395996,0.055571


In [19]:
ethnicity_percentages = pd.read_csv("ethnicity_percentages.csv")
ethnicity_percentages.head()

,State Name,White Percentage,Black or African American Percentage,American Indian and Alaska Native Percentage,Native Hawaiian and Other Pacific Islander Percentage,Other Percentage
0,Alabama,0.696713,0.270303,0.005271,0.000465,0.014058
1,Alaska,0.713131,0.035070,0.155352,0.013427,0.015543
2,Arizona,0.803048,0.044557,0.045676,0.002033,0.072212
3,Arkansas,0.792169,0.158276,0.006702,0.002430,0.025844
4,California,0.635489,0.060924,0.007861,0.004092,0.143478


In [20]:
ethnicity_merge = pd.merge(State_FIPS_pd, ethnicity_percentages, how="right", left_on="State Name", right_on="State Name")
ethnicity_merge.head()

,State FIPS,State Name,White Percentage,Black or African American Percentage,American Indian and Alaska Native Percentage,Native Hawaiian and Other Pacific Islander Percentage,Other Percentage
0,1.0,Alabama,0.696713,0.270303,0.005271,0.000465,0.014058
1,2.0,Alaska,0.713131,0.035070,0.155352,0.013427,0.015543
2,4.0,Arizona,0.803048,0.044557,0.045676,0.002033,0.072212
3,5.0,Arkansas,0.792169,0.158276,0.006702,0.002430,0.025844
4,6.0,California,0.635489,0.060924,0.007861,0.004092,0.143478


In [21]:
income = pd.read_csv("income_median_age.csv")
income.head()

,Unnamed: 0,State Name,Household Income,Median Age,Median Male Age,Median Female Age
0,0,Alabama,40271.40,40.45,38.87,41.98
1,1,Alaska,66425.45,36.94,37.60,36.16
2,2,Arizona,45816.67,39.93,38.91,41.13
3,3,Arkansas,39025.08,41.33,39.95,42.68
4,4,California,61046.76,39.47,38.45,40.81


In [22]:
income_merge = pd.merge(State_FIPS_pd, income, how="right", left_on="State Name", right_on="State Name")
income_merge.head()

,State FIPS,State Name,Unnamed: 0,Household Income,Median Age,Median Male Age,Median Female Age
0,1,Alabama,0,40271.40,40.45,38.87,41.98
1,2,Alaska,1,66425.45,36.94,37.60,36.16
2,4,Arizona,2,45816.67,39.93,38.91,41.13
3,5,Arkansas,3,39025.08,41.33,39.95,42.68
4,6,California,4,61046.76,39.47,38.45,40.81


In [23]:
total_data = pd.read_csv("final_df.csv")
total_data.head()

,State Name,Household Income,Median Age,Median Male Age,Median Female Age,No College Degree,No College Degree Percentage,Associates or Bachelors Degree,Associates or Bachelors Degree Percentage,Post Graduate Degree,...,Black or African American,Black or African American Percentage,American Indian and Alaska Native,American Indian and Alaska Native Percentage,Asian,Asian Percentage,Native Hawaiian and Other Pacific Islander,Native Hawaiian and Other Pacific Islander Percentage,Other,Other Percentage
0,Alabama,"$40,271.40",40.45,38.87,41.98,1567420.0,64.71%,771175.0,31.84%,83647.0,...,1287167.0,27.03%,25098.0,0.53%,62815.0,1.32%,2213.0,0.05%,66942.0,1.41%
1,Alaska,"$66,425.45",36.94,37.60,36.16,229333.0,61.72%,128518.0,34.59%,13743.0,...,23702.0,3.51%,104995.0,15.54%,45604.0,6.75%,9075.0,1.34%,10505.0,1.55%
2,Arizona,"$45,816.67",39.93,38.91,41.13,1960152.0,59.68%,1189412.0,36.21%,134913.0,...,292831.0,4.46%,300182.0,4.57%,213417.0,3.25%,13364.0,0.20%,474583.0,7.22%
3,Arkansas,"$39,025.08",41.33,39.95,42.68,1017698.0,68.94%,412898.0,27.97%,45657.0,...,459969.0,15.83%,19476.0,0.67%,42368.0,1.46%,7062.0,0.24%,75106.0,2.58%
4,California,"$61,046.76",39.47,38.45,40.81,10129401.0,54.84%,7313928.0,39.60%,1026371.0,...,2263222.0,6.09%,292018.0,0.79%,5503672.0,14.82%,152027.0,0.41%,5329952.0,14.35%


In [24]:
gmaps.configure(api_key=gkey)

In [25]:
locations = education_merge[["State FIPS"]].astype(float)
Education = education_merge[["No College Degree Percentage", "Associates or Bachelors Degree Percentage", "Post Graduate Degree Percentage"]].astype(float)
Education.head()

,No College Degree Percentage,Associates or Bachelors Degree Percentage,Post Graduate Degree Percentage
0,0.647095,0.318372,0.034533
1,0.617160,0.345856,0.036984
2,0.596793,0.362131,0.041076
3,0.689379,0.279693,0.030928
4,0.548433,0.395996,0.055571


In [28]:

state = []
lats_lngs = []
state_df = [["State"]["Lats and Lngs"]])

SyntaxError: invalid syntax (<ipython-input-28-7d28db434cff>, line 3)

In [12]:
gmaps.figure(display_toolbar=True, display_errors=True, zoom_level=None, tilt=45, center=None, layout=None, map_type='ROADMAP', mouse_handling='COOPERATIVE')

TypeError: figure() got an unexpected keyword argument 'tilt'

In [13]:
fig = gmaps.figure()
drawing = gmaps.drawing_layer(features=[
    gmaps.Line((46.23, 5.86), (46.44, 5.24), stroke_weight=3.0),
    gmaps.Marker((46.88, 5.45), label='D'),
    gmaps.Polygon(
        [(46.72, 6.06), (46.48, 6.49), (46.79, 6.91)],
        fill_color='red'
    )
])
fig.add_layer(drawing)
fig

Figure(layout=FigureLayout(height='420px'))